<a href="https://colab.research.google.com/github/azizamirsaidova/fake-news-detection/blob/main/notebooks/pre_trained_transformer_models_for_text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langdetect

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 981 kB 13.7 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=985e97853530238cb3a54700f75325072d133c3b466ba48a22afa24eeb0cc33e
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect


In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import string
import nltk
from nltk import PorterStemmer
import re
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english')
from gensim import corpora
import gensim
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [2]:
uri_train  = 'https://raw.githubusercontent.com/azizamirsaidova/fake-news-detection/main/liar-dataset/train.tsv'
uri_valid  = 'https://raw.githubusercontent.com/azizamirsaidova/fake-news-detection/main/liar-dataset/valid.tsv'
uri_test  = 'https://raw.githubusercontent.com/azizamirsaidova/fake-news-detection/main/liar-dataset/test.tsv'

    
df_train = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

    
df_valid = pd.read_table(uri_valid,
                             names =['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])


df_test = pd.read_csv(uri_test, sep='\t', 
                            names =['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue']) 


df = pd.concat([df_train, df_valid])

In [4]:
dff = df[['label', 'statement']]

**Label:**

half-true - 2362

**false** - 2258

mostly-true - 2213

barely-true - 1891

**true** - 1845

pants-fire - 955


In [5]:
false = dff[['statement']].loc[df['label'] == 'false']

In [6]:
true = dff[['statement']].loc[df['label'] == 'true']

In [7]:
false.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2258 entries, 0 to 1282
Data columns (total 1 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   statement  2258 non-null   object
dtypes: object(1)
memory usage: 35.3+ KB


In [8]:
false['statement']= false['statement'].astype(str)

In [9]:
true['statement'] = true['statement'].astype(str)

## **Generate True News using GPT3**

In [10]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |█████████████▌                  | 834.1 MB 1.8 MB/s eta 0:10:33tcmalloc: large alloc 1147494400 bytes == 0x38f9c000 @  0x7f8260b08615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████████████████               | 1055.7 MB 1.3 MB/s eta 0:11:37tcmalloc: large alloc 1434370048 bytes == 0x7d5f2000 @  0x7f8260b08615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |█████

In [11]:
from transformers import pipeline

In [12]:
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-1.3B')

Downloading:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/200 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

In [13]:
only_10 = true.head(10)

In [14]:
only_10 = list(only_10['statement'])

In [15]:
only_10

['The Chicago Bears have had more starting quarterbacks in the last 10 years than the total number of tenured (UW) faculty fired during the last two decades.',
 "McCain opposed a requirement that the government buy American-made motorcycles. And he said all buy-American provisions were quote 'disgraceful.' ",
 'Says Paul Ryan is still endorsing Trump.',
 'We have a federal government that thinks they have the authority to regulate our toilet seats.',
 'Austin is a city that has basically doubled in size every 25 years or so since it was founded.',
 "The nuclear test conducted in our nation this time is the Earth's 2,054th nuclear test. The five permanent members of the U.N. Security Council have conducted 99.99 percent of the total nuclear tests.",
 'Under President George W. Bush, we added $4.9 trillion to the debt. Under President Obama weve added $6.5 trillion to the debt.',
 'Hillary Clinton in 2005 co-sponsored legislation that would jail flag burners.',
 'We created 800,000 new j

In [ ]:
only_10_true_result = generator(only_10, max_length=200, do_sample=True, temperature=5.0, top_k = 50)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [17]:
only_10_true_result

[[{'generated_text': 'The Chicago Bears have had more starting quarterbacks in the last 10 years than the total number of tenured (UW) faculty fired during the last two decades. It takes a tremendous amount of talent to be the best passer in the league and the last'}],
 [{'generated_text': 'McCain opposed a requirement that the government buy American-made motorcycles. And he said all buy-American provisions were quote \'disgraceful.\' \n\n"He said it was disgraceful to require the government to buy American-made motorcycles'}],
 [{'generated_text': 'Says Paul Ryan is still endorsing Trump. “I’ve been for him since the beginning,” he offered.\n\n“That’s just a question of perspective,” said former Republican Senate leader Tip O'}],
 [{'generated_text': 'We have a federal government that thinks they have the authority to regulate our toilet seats.\n\nAnd now, they have the authority to legislate how we use our private restrooms.\n\nCongressman Mike Kelly (D-IL) wants to do'}],
 [{'gener

## **Distilled BERT**

In [12]:
from transformers import DistilBertTokenizer, DistilBertModel
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
# text = true['statement']
encoded_input = tokenizer(only_10, return_tensors='pt', padding=True)
output = model(**encoded_input)

In [22]:
last_hidden_states = output.last_hidden_state

In [ ]:
last_hidden_states

## **GPT2**

In [ ]:
!pip install transformers

In [11]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# initialize tokenizer and model from pretrained GPT2 model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

In [13]:
only_10 = true.head(10)
only_10 = list(only_10['statement'].astype(str))

In [41]:
inputs = tokenizer.encode(only_10, return_tensors='pt')
outputs = model.generate(inputs, max_length=200, do_sample=True, temperature=5.0, top_k=50)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [43]:
text = tokenizer.decode(outputs[0], skip_special_tokens=True)

In [39]:
only_10

['The Chicago Bears have had more starting quarterbacks in the last 10 years than the total number of tenured (UW) faculty fired during the last two decades.',
 "McCain opposed a requirement that the government buy American-made motorcycles. And he said all buy-American provisions were quote 'disgraceful.' ",
 'Says Paul Ryan is still endorsing Trump.',
 'We have a federal government that thinks they have the authority to regulate our toilet seats.',
 'Austin is a city that has basically doubled in size every 25 years or so since it was founded.',
 "The nuclear test conducted in our nation this time is the Earth's 2,054th nuclear test. The five permanent members of the U.N. Security Council have conducted 99.99 percent of the total nuclear tests.",
 'Under President George W. Bush, we added $4.9 trillion to the debt. Under President Obama weve added $6.5 trillion to the debt.',
 'Hillary Clinton in 2005 co-sponsored legislation that would jail flag burners.',
 'We created 800,000 new j

In [44]:
text

"We'll talk over all eight films coming in between now. The film will have only 1 studio involved so that it remains fair before all those extras on tour have already had theirs screened on film at these very expensive screening theatros. Plus he may add 'A Little Pigeona and No Life will still work in London but would come less so when compared to a box Office debut by Denge Toussuite films or Universal-made Universal releases with 2 to 3 hours total run, or the very long running of many theatricals in Los Anges until 2016 before then even starting working a home release. After 20 or as much casting call he would begin planning this and more and more work that goes onto to complete your franchise. Of my current number a few will start around 2016 and as it seems for us there will really no problem with just having at least ONE in Hollywood next year to move beyond, this also looks pretty obvious because what we"